## Этап 1. Получение данных

Импортируем библиотеки.

Прочитаем файл с данными и сохраним его в переменной data.

In [1]:
import pandas as pd

data = pd.read_csv('/datasets/data.csv')

Получение первых 20 строк таблицы.

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Общая информация о данных таблицы.

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Этап 2. Предобработка данных

### Удаление пропусков

Проверим данные на наличие пропусков вызовом набора методов для суммирования пропущенных значений.

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце обработаем на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполним пропуски в этом столбце медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` заполним медианным доходом среди всех записей с тем же типом.

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

Заменим все отрицательные значения количество дней трудового стажа в столбце `days_employed` положительными с помощью метода `abs()`.

In [6]:
data['days_employed'] = data['days_employed'].abs()

Найдём медианное значение трудового стажа `days_employed` в днях.

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

Выведем перечень уникальных значений столбца `children`.

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Удалим строки, в которых встречаются аномальные значения столбца `children`.

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Проверим столбец `children` на наличие аномальных значений.

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Убедимся, что все пропуски заполнены.

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

Заменим тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Обработаем неявные дубликаты в столбце `education`.

In [14]:
data['education'] = data['education'].str.lower()

Выведем на экран количество строк-дубликатов в данных, удалим их.

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

Добавим столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Выведем на экран перечень уникальных целей взятия кредита.

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` присвоит записи одну из категорий:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Этап 3. Исследование данных

Выведем на экран долю должников в зависимости от количества детей в семье.

In [22]:
children_debt = data[['children', 'debt']] # выбрали нужные столбцы
children_value_counts = children_debt['children'].value_counts() # узнали количсетво наблюдений для каждого размера семьи
children_debt_grouped = children_debt.groupby('children').sum() # узнали количество задолжностей для каждого размера семьи
children_debt_ratio = children_debt_grouped['debt'] / children_value_counts # посчитали процент должников для каждого размера семьи

for i in range(len(children_debt_ratio)):
    print(f'Доля должников среди семей с количеством детей {i} - {children_debt_ratio[i]:.2%} при количестве наблюдений {children_value_counts[i]}')

Доля должников среди семей с количеством детей 0 - 7.54% при количестве наблюдений 14091
Доля должников среди семей с количеством детей 1 - 9.23% при количестве наблюдений 4808
Доля должников среди семей с количеством детей 2 - 9.45% при количестве наблюдений 2052
Доля должников среди семей с количеством детей 3 - 8.18% при количестве наблюдений 330
Доля должников среди семей с количеством детей 4 - 9.76% при количестве наблюдений 41
Доля должников среди семей с количеством детей 5 - 0.00% при количестве наблюдений 9


Количетво наблюдений для семей с 3-мя и более детьми может быть признано недостаточным для достоверной оценки. Для семей с 0-2 детьми наблюдается прямая корреляция между колчиеством детей и вероятностью просрочить кредит.

Напишем функцию, которая выведет на экран долю должников в зависимости от семейного положения.

In [46]:
def dependency_search(parameter):
    data_pivot = data.pivot_table(index=parameter, values='debt', aggfunc=['sum', 'count']) # собрали сводную таблицу
    data_pivot['debtors'] = data_pivot['sum'] / data_pivot['count'] # нашли процент должников
    data_pivot.rename(columns = {'debt' : '', 'sum' : 'Количество должников', 'count' : 'Всего клиентов', 'debtors' : 'Доля должников'}, inplace=True) # навели красоту
    return data_pivot

In [47]:
dependency_search('family_status')

,Количество должников,Всего клиентов,Доля должников
,,,
family_status,,,
Не женат / не замужем,273,2796,0.097639
в разводе,84,1189,0.070648
вдовец / вдова,63,951,0.066246
гражданский брак,385,4134,0.093130
женат / замужем,927,12261,0.075606


Судя по результатам, лица, которые никогда официально не регистрировали свои отношения, с большей вероятностью просрочат кредит.

Выведем на экран долю должников в зависимости от уровня дохода.

In [45]:
dependency_search('total_income_category')

,Количество должников,Всего клиентов,Доля должников
,,,
total_income_category,,,
A,2,25,0.080000
B,354,5014,0.070602
C,1353,15921,0.084982
D,21,349,0.060172
E,2,22,0.090909


Если учитывать категории с выборкой меньше 1000 наблюдений, зависимость между уровнем дохода и возвратом кредита в срок не наблюдается. В противном случае данные говорят о том, что группы с меньшим доходом с большей вероятностью просрочат кредит.

Выведем на экран долю должников в зависимости от цели кредита.

In [44]:
dependency_search('purpose_category')

,Количество должников,Всего клиентов,Доля должников
,,,
purpose_category,,,
операции с автомобилем,400,4279,0.093480
операции с недвижимостью,780,10751,0.072551
получение образования,369,3988,0.092528
проведение свадьбы,183,2313,0.079118


Выдавать кредит на операции с недвижимостью и проведение свадьбы безопаснее, чем на операции с автомобилем и получение образования.

### Этап 4. Результаты исследования

Поставленная задача: Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

Вывод: Семейное положение и количество детей клиента влияет на факт погашения кредита в срок - клиенты, официально состоящие/состоявшие в отношениях и без детей, с меньшей вероятностью просрочат кредит. Также можно учитывать уровень дохода и цель кредита, клиенты с высоким доходом и собирающиеся производить операции с недвижимостью наиболее надёжные.